In [1]:
import pandas as pd

df_or = pd.read_csv("Y_0.csv")

df_or['is_malware'] = df_or['is_malware'].astype(bool)
df_or['is_dga'] = df_or['is_dga'].astype(bool)

df_or['legit'] = (df_or['is_malware'] == 0) & (df_or['is_dga'] == 0)

df_or['not_dga'] = (df_or['is_malware'] == True) & (df_or['is_dga'] == False)

df_or

,id,is_malware,is_dga,Y_none,Y_tld,Y_icann,Y_private,Y_none_first_17char,Y_tld_first_17char,Y_icann_first_17char,Y_private_first_17char,legit,not_dga
0,0,False,False,1.005598e-12,1.556818e-02,2.598478e-04,6.312761e-02,4.698381e-08,1.752473e-03,3.849515e-04,2.161021e-06,True,False
1,1,False,False,6.368934e-09,4.435358e-08,8.998251e-09,4.551743e-05,1.545209e-09,3.403100e-06,2.971966e-07,1.740100e-06,True,False
2,2,False,False,1.234430e-08,1.274059e-04,1.402510e-06,8.551755e-03,1.695813e-04,1.991690e-05,1.418202e-08,3.691444e-07,True,False
3,3,False,False,5.956628e-07,3.501129e-16,8.098094e-08,7.502151e-08,4.999577e-09,4.183368e-14,1.145445e-09,1.677584e-05,True,False
4,4,False,False,6.954845e-10,2.457577e-18,5.336343e-06,2.167046e-08,1.455787e-09,9.327148e-15,6.141904e-10,1.078596e-08,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438265,438265,True,False,2.705789e-08,1.399286e-02,3.384466e-08,8.182876e-02,9.702229e-01,1.399286e-02,3.384466e-08,8.182876e-02,False,True
438266,438266,True,False,7.792556e-12,1.808250e-08,7.530522e-08,1.851928e-06,2.799801e-12,1.808250e-08,7.530522e-08,1.851928e-06,False,True
438267,438267,True,False,7.774512e-01,1.189371e-01,2.149270e-02,4.643767e-01,9.999257e-01,1.189371e-01,2.149270e-02,4.643767e-01,False,True
438268,438268,True,False,9.999977e-01,9.999644e-01,9.799217e-01,8.686514e-01,9.997970e-01,9.999644e-01,9.799217e-01,8.686514e-01,False,True


In [8]:
df_X = pd.read_csv("X.csv")

df_X

df_or['dn'] = df_X['none']

df_or['not_dga'].value_counts(), df_or['legit'].value_counts()

(False    251314
 True     186956
 Name: not_dga, dtype: int64,
 False    238270
 True     200000
 Name: legit, dtype: int64)

In [2]:
th = 0.5


for tmp in ['is_malware', 'is_dga', 'not_dga']:

    df = df_or.copy()

    df = df[(df.legit) | (df[tmp])]


    df['good'] = df['legit']
    df['bad'] = df[tmp]


    def cm(row, col):
        if row[col] > 0.5 and row['bad']: return 'tp'
        if row[col] <= 0.5 and row['bad']: return 'fp'
        if row[col] > 0.5 and row['good']: return 'fn'
        if row[col] <= 0.5 and row['good']: return 'tn'
        return 'err'


    results = []
    for col in 'Y_none,Y_tld,Y_icann,Y_private,Y_none_first_17char,Y_tld_first_17char,Y_icann_first_17char,Y_private_first_17char'.split(','):
        s_p = df[col] > th

        n = (df['good']).sum()
        p = (df['bad']).sum()

        # print(df.apply(cm, axis=1, col=col))

        tp = (s_p & (df['bad']))
        tn = ((~s_p) & (df['good']))
        fp = ((s_p) & (df['good']))
        fn = ((~s_p) & (df['bad']))

        df['cm'] = ''

        df['cm'] = df['cm'].where(~tp, 'tp')
        df['cm'] = df['cm'].where(~fn, 'fn')
        df['cm'] = df['cm'].where(~fp, 'fp')
        df['cm'] = df['cm'].where(~tn, 'tn')

#         print(f"{col}")
#         print(f"TPR:\t{tp.sum()}\t{tp.sum()/p:6.2%}")
#         print(f"FNR:\t{fn.sum()}\t{fn.sum()/p:02.2%}")
#         print(f"TNR:\t{tn.sum()}\t{tn.sum()/n:2.2%}")
#         print(f"FPR:\t{fp.sum()}\t{fp.sum()/n:2.2%}\n")

        results.append([
            col[2:],

            tp.sum(),
            fn.sum(),
            tn.sum(),
            fp.sum(),

            tp.sum() / p,
            fn.sum() / p,
            tn.sum() / n,
            fp.sum() / n,

            tp.sum()/fp.sum()
        ])

        pass

    print(tmp)
    print(pd.DataFrame(results, columns=['nn', 'tp', 'fn', 'tn', 'fp', 'tp%', 'fn%', 'tn%', 'fp%', 'veri/falsi']).to_csv(index=False))



is_malware
nn,tp,fn,tn,fp,tp%,fn%,tn%,fp%,veri/falsi
none,69559,168711,198513,1487,0.2919335207957359,0.708066479204264,0.992565,0.007435,46.778076664425015
tld,55960,182310,199034,966,0.2348596130440257,0.7651403869559743,0.99517,0.00483,57.9296066252588
icann,57529,180741,199064,936,0.24144457967851596,0.758555420321484,0.99532,0.00468,61.462606837606835
private,57029,181241,198929,1071,0.2393461199479582,0.7606538800520418,0.994645,0.005355,53.248366013071895
none_first_17char,71754,166516,193748,6252,0.30114575901288454,0.6988542409871155,0.96874,0.03126,11.476967370441459
tld_first_17char,48223,190047,198451,1549,0.20238804717337475,0.7976119528266252,0.992255,0.007745,31.131697869593285
icann_first_17char,48456,189814,198935,1065,0.20336592940781467,0.7966340705921854,0.994675,0.005325,45.498591549295774
private_first_17char,54587,183683,197595,2405,0.22909724262391404,0.7709027573760859,0.987975,0.012025,22.697297297297297

is_dga
nn,tp,fn,tn,fp,tp%,fn%,tn%,fp%,veri/falsi
none,4

In [7]:
th = 0.5


df = df_or.copy()


df = df[(df.legit) | ((df.is_malware == 1) & (df.is_dga == 0))]


df['good'] = df['legit']
df['bad'] = df['is_dga']


def cm(row, col):
    if row[col] > 0.5 and row['bad']: return 'tp'
    if row[col] <= 0.5 and row['bad']: return 'fp'
    if row[col] > 0.5 and row['good']: return 'fn'
    if row[col] <= 0.5 and row['good']: return 'tn'
    return 'err'


results = []
for col in 'Y_none,Y_tld,Y_icann,Y_private,Y_none_first_17char,Y_tld_first_17char,Y_icann_first_17char,Y_private_first_17char'.split(','):
    s_p = df[col] > th
    
    n = (df['good']).sum()
    p = (df['bad']).sum()
    
    # print(df.apply(cm, axis=1, col=col))

    tp = (s_p & (df['bad']))
    tn = ((~s_p) & (df['good']))
    fp = ((s_p) & (df['good']))
    fn = ((~s_p) & (df['bad']))
    
    df['cm'] = ''
    
    df['cm'] = df['cm'].where(~tp, 'tp')
    df['cm'] = df['cm'].where(~fn, 'fn')
    df['cm'] = df['cm'].where(~fp, 'fp')
    df['cm'] = df['cm'].where(~tn, 'tn')
    
    print(f"{col}")
    print(f"TPR:\t{tp.sum()}\t{tp.sum()/p:6.2%}")
    print(f"FNR:\t{fn.sum()}\t{fn.sum()/p:02.2%}")
    print(f"TNR:\t{tn.sum()}\t{tn.sum()/n:2.2%}")
    print(f"FPR:\t{fp.sum()}\t{fp.sum()/n:2.2%}\n")
    
    results.append([
        col[2:],
        tp.sum() / p,
        fp.sum() / n,
        fn.sum() / p,
        tn.sum() / n,
        tp.sum()/fp.sum()
    ])
    # display(df[df['cm'] == 'tn'])
    
    pass

print(pd.DataFrame(results, columns=['nn', 'tp', 'fp', 'fn', 'tn', 'veri/falsi']).to_markdown(index=False))

print(df.shape[0])

Y_none
TPR:	0	  nan%
FNR:	0	nan%
TNR:	198513	99.26%
FPR:	1487	0.74%

Y_tld
TPR:	0	  nan%
FNR:	0	nan%
TNR:	199034	99.52%
FPR:	966	0.48%

Y_icann
TPR:	0	  nan%
FNR:	0	nan%
TNR:	199064	99.53%
FPR:	936	0.47%

Y_private
TPR:	0	  nan%
FNR:	0	nan%
TNR:	198929	99.46%
FPR:	1071	0.54%

Y_none_first_17char
TPR:	0	  nan%
FNR:	0	nan%
TNR:	193748	96.87%
FPR:	6252	3.13%

Y_tld_first_17char
TPR:	0	  nan%
FNR:	0	nan%
TNR:	198451	99.23%
FPR:	1549	0.77%

Y_icann_first_17char
TPR:	0	  nan%
FNR:	0	nan%
TNR:	198935	99.47%
FPR:	1065	0.53%



<ipython-input-7-c62b74970be5>:44: RuntimeWarning: invalid value encountered in long_scalars
  print(f"TPR:\t{tp.sum()}\t{tp.sum()/p:6.2%}")
<ipython-input-7-c62b74970be5>:45: RuntimeWarning: invalid value encountered in long_scalars
  print(f"FNR:\t{fn.sum()}\t{fn.sum()/p:02.2%}")
<ipython-input-7-c62b74970be5>:51: RuntimeWarning: invalid value encountered in long_scalars
  tp.sum() / p,
<ipython-input-7-c62b74970be5>:53: RuntimeWarning: invalid value encountered in long_scalars
  fn.sum() / p,
<ipython-input-7-c62b74970be5>:44: RuntimeWarning: invalid value encountered in long_scalars
  print(f"TPR:\t{tp.sum()}\t{tp.sum()/p:6.2%}")
<ipython-input-7-c62b74970be5>:45: RuntimeWarning: invalid value encountered in long_scalars
  print(f"FNR:\t{fn.sum()}\t{fn.sum()/p:02.2%}")
<ipython-input-7-c62b74970be5>:51: RuntimeWarning: invalid value encountered in long_scalars
  tp.sum() / p,
<ipython-input-7-c62b74970be5>:53: RuntimeWarning: invalid value encountered in long_scalars
  fn.sum() / p

Y_private_first_17char
TPR:	0	  nan%
FNR:	0	nan%
TNR:	197595	98.80%
FPR:	2405	1.20%

| nn                   |   tp |       fp |   fn |       tn |   veri/falsi |
|:---------------------|-----:|---------:|-----:|---------:|-------------:|
| none                 |  nan | 0.007435 |  nan | 0.992565 |            0 |
| tld                  |  nan | 0.00483  |  nan | 0.99517  |            0 |
| icann                |  nan | 0.00468  |  nan | 0.99532  |            0 |
| private              |  nan | 0.005355 |  nan | 0.994645 |            0 |
| none_first_17char    |  nan | 0.03126  |  nan | 0.96874  |            0 |
| tld_first_17char     |  nan | 0.007745 |  nan | 0.992255 |            0 |
| icann_first_17char   |  nan | 0.005325 |  nan | 0.994675 |            0 |
| private_first_17char |  nan | 0.012025 |  nan | 0.987975 |            0 |
386956


<ipython-input-7-c62b74970be5>:44: RuntimeWarning: invalid value encountered in long_scalars
  print(f"TPR:\t{tp.sum()}\t{tp.sum()/p:6.2%}")
<ipython-input-7-c62b74970be5>:45: RuntimeWarning: invalid value encountered in long_scalars
  print(f"FNR:\t{fn.sum()}\t{fn.sum()/p:02.2%}")
<ipython-input-7-c62b74970be5>:51: RuntimeWarning: invalid value encountered in long_scalars
  tp.sum() / p,
<ipython-input-7-c62b74970be5>:53: RuntimeWarning: invalid value encountered in long_scalars
  fn.sum() / p,


In [3]:
df

,id,is_malware,is_dga,Y_none,Y_tld,Y_icann,Y_private,Y_none_first_17char,Y_tld_first_17char,Y_icann_first_17char,Y_private_first_17char,legit,not_dga,good,bad,cm
0,0,False,False,1.005598e-12,1.556818e-02,2.598478e-04,6.312761e-02,4.698381e-08,1.752473e-03,3.849515e-04,2.161021e-06,True,False,True,False,tn
1,1,False,False,6.368934e-09,4.435358e-08,8.998251e-09,4.551743e-05,1.545209e-09,3.403100e-06,2.971966e-07,1.740100e-06,True,False,True,False,tn
2,2,False,False,1.234430e-08,1.274059e-04,1.402510e-06,8.551755e-03,1.695813e-04,1.991690e-05,1.418202e-08,3.691444e-07,True,False,True,False,tn
3,3,False,False,5.956628e-07,3.501129e-16,8.098094e-08,7.502151e-08,4.999577e-09,4.183368e-14,1.145445e-09,1.677584e-05,True,False,True,False,tn
4,4,False,False,6.954845e-10,2.457577e-18,5.336343e-06,2.167046e-08,1.455787e-09,9.327148e-15,6.141904e-10,1.078596e-08,True,False,True,False,tn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438265,438265,True,False,2.705789e-08,1.399286e-02,3.384466e-08,8.182876e-02,9.702229e-01,1.399286e-02,3.384466e-08,8.182876e-02,False,True,False,True,fn
438266,438266,True,False,7.792556e-12,1.808250e-08,7.530522e-08,1.851928e-06,2.799801e-12,1.808250e-08,7.530522e-08,1.851928e-06,False,True,False,True,fn
438267,438267,True,False,7.774512e-01,1.189371e-01,2.149270e-02,4.643767e-01,9.999257e-01,1.189371e-01,2.149270e-02,4.643767e-01,False,True,False,True,fn
438268,438268,True,False,9.999977e-01,9.999644e-01,9.799217e-01,8.686514e-01,9.997970e-01,9.999644e-01,9.799217e-01,8.686514e-01,False,True,False,True,tp
